# How to build a simple LLM App with LangChain and deploy it with LangServe
* Very simple LLM App.
* Goal of the App: translate text from English into another language.
* Second version: how to build this app using a language other than English.

## Concepts included
* Monitor with LangSmith.
* Connect with a LLM.
* Use a Prompt Template.
* Use an Output Parser.
* Chain the Prompt Template, the LLM call and the Output Parser.
* Deploy with LangServe.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simpleTranslator.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-simpleTranslator**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install python-dotenv

In [26]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [27]:
#!pip install langchain

## Connect with an LLM and ask it to translate  a sentence from English to Spanish

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [28]:
#!pip install langchain-openai

#### OpenAI LLM options
* See [OpenAI website](https://platform.openai.com/docs/models/gpt-3-5-turbo).
* For this project, we will use gpt-3.5-turbo

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

* System Message: defines the role played by the LLM.
* Human Message: the user input.

In [30]:
from langchain_core.messages import HumanMessage, SystemMessage

messagesToTheLLM = [
    SystemMessage(content="Translate the following from English into Spanish"),
    HumanMessage(content="Generative AI is the greatest value-creation opportunity in Human History."),
]

#### Step 1: call the LLM

* `invoke` is the most basic way to call the LLM. We will see more ways of doing it soon.

In [31]:
llm.invoke(messagesToTheLLM)

AIMessage(content='La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia humana.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 32, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-39ae644f-4c3a-4c87-897d-34631695fb86-0')

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Use an Output Parser to format the response of the LLM as a string of text

In [32]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

#### Step 1: call the LLM

In [33]:
initialResponse = llm.invoke(messagesToTheLLM)

#### Step 2: apply the parser to format the output

In [34]:
formattedResponse = parser.invoke(initialResponse)

In [35]:
print(formattedResponse)

La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia humana.


## Chain the LLM Call and the Output Parser

In [36]:
llmAndParserChained = llm | parser

#### Steps 1 and 2: call the LLM and apply the parser to the output

In [37]:
responseFromChain=llmAndParserChained.invoke(messagesToTheLLM)

In [38]:
print(responseFromChain)

La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia humana.


## Use a Prompt Template with a System Message and Variables
* variables:
    * text_input: the text the user wants to translate 
    * language: the language the user wants to translate the text_input into.

In [39]:
from langchain_core.prompts import ChatPromptTemplate

In [43]:
system_template = "Translate the following text input into {language}:"

In [46]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        ("user", "{text_input}")
    ]
)

In [58]:
SecondChain = prompt_template | llm | parser

In [59]:
ResponseFromTheLLM = SecondChain.invoke(
    {
    "language": "Spanish",
    "text_input": "Now is the moment to learn Generative AI!"
    }
)

In [60]:
print(ResponseFromTheLLM)

¡Ahora es el momento de aprender la IA Generativa!


## Deploy the App using LangServe

**LangServe** is a component of the LangChain framework designed to help developers deploy language model applications as web services easily and efficiently. In simpler terms, it turns your language-based applications (like chatbots, translators, or any other app using language models) into web services that can be accessed over the internet through URLs.

Here’s a step-by-step explanation on how to deploy a LangChain app using LangServe:

1. **Develop Your App**: First, you create your language model application using LangChain. This involves setting up the language model, defining how it should process inputs and outputs, and possibly integrating with other systems or APIs.

2. **Prepare for Deployment**:
   - Ensure that your app is well-tested locally and behaves as expected.
   - Configure any necessary environment variables, such as API keys or service endpoints, which the app will need to function properly once deployed.

3. **Set Up LangServe**:
   - LangServe acts as a server environment for your LangChain app. You'll need to configure LangServe settings specific to your application, such as the port number on which the server should run and any specific route paths (URLs) that should be handled by your app.

4. **Deploy the Application**:
   - Using tools and commands provided by LangChain, you can start the LangServe server with your application loaded on it. This makes your app available on a specified port of your server machine.
   - If you need your service to be accessible publicly over the internet (not just locally), you might need to deploy it on a cloud platform or a dedicated server with a public IP address.

5. **Access the App via URLs**:
   - Once deployed, your application can be accessed through web URLs. This could be for direct user interaction (like a web-based chat interface) or for integration with other systems (like APIs that other software can call to use your app).

LangServe effectively simplifies the process of making your LangChain apps accessible as standard web services, which can be a powerful way to deploy AI-driven language applications.

**You will need to install langserve from terminal**:

In [61]:
#!pip install "langserve[all]"

* See the file 001-simpleTranslator.py in your code editor.
* Remember to create .gitignore file and include .env there.

In [64]:
# from fastapi import FastAPI
# from langserve import add_routes
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from langchain_openai import ChatOpenAI
# from dotenv import load_dotenv, find_dotenv

# import os

# _ = load_dotenv(find_dotenv())

# openai_api_key = os.environ["OPENAI_API_KEY"]

# llm = ChatOpenAI(model="gpt-3.5-turbo")

# parser = StrOutputParser()

# system_template = "Translate the following into {language}:"

# prompt_template = ChatPromptTemplate.from_messages([
#     ('system', system_template),
#     ('user', '{text}')
# ])

# chain = prompt_template | llm | parser

# app = FastAPI(
#   title="simpleTranslator",
#   version="1.0",
#   description="A simple API server using LangChain's Runnable interfaces",
# )

# add_routes(
#     app,
#     chain,
#     path="/chain",
# )

# if __name__ == "__main__":
#     import uvicorn

#     uvicorn.run(app, host="localhost", port=8000)

#### Run the server
* In terminal:
    * python 001-simpleTranslator.py
* Check the app in the browser:
    * [http://localhost:8000/](http://localhost:8000/)
    * Nothing fancy there... yet.

#### Now let's try the app using the "toy demo UI" provided by LangServe Playground
* Try the app in the browser:
    * [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/) 

#### And finally, let's see how we can use the LangServe Playground from the notebook

In [63]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "Spanish", "text": "Generative AI is a bigger opportunity than Internet"})

'La inteligencia artificial generativa es una oportunidad más grande que Internet.'

## LangServe vs. FastAPI
Deploying a LangChain app with LangServe or FastAPI involves similar basic principles—both methods aim to serve your application over the web—but they differ in their level of specialization and the features they offer. Here's a simple explanation of the main differences between these two deployment options:

#### FastAPI

1. **General-Purpose Framework**: FastAPI is a modern, fast (high-performance) web framework for building APIs with Python. It's designed to be simple to use but powerful in capabilities, supporting the development of robust APIs and web applications.

2. **Flexibility**: FastAPI provides extensive flexibility in how you structure your application. It allows for detailed customization of request handling, response formatting, and middleware integration, making it suitable for a wide variety of web services beyond just language applications.

3. **Manual Setup**: When deploying a LangChain app with FastAPI, you need to manually set up the routing, request handling, and integration with LangChain. This involves writing more boilerplate code and handling more configuration details.

4. **Community and Ecosystem**: FastAPI has a large developer community and a rich ecosystem of plugins and tools, which can be advantageous for solving common web development problems and integrating with other technologies.

#### LangServe

1. **Specialized for LangChain**: LangServe is tailored specifically for deploying LangChain applications. This specialization means it comes with built-in configurations and setups optimized for language model applications, reducing the need to manually configure many aspects of deployment.

2. **Simplicity and Convenience**: LangServe aims to simplify the process of turning your LangChain model into a web service. It abstracts away many of the lower-level details of web service configuration, allowing you to focus more on developing the language model itself.

3. **Integrated Tools**: Since LangServe is designed to work seamlessly with LangChain, it often includes tools and features that specifically support language model operations, such as handling different types of language inputs and outputs more effectively.

4. **Limited Flexibility**: While offering simplicity, LangServe may not provide as much flexibility as FastAPI in terms of general web development capabilities. It's optimized for a specific type of application, which might limit its utility outside of deploying language models.

### Summary

- **Choose FastAPI** if you need a more flexible, general-purpose approach that can handle a wide variety of web services, require detailed customization, or want to integrate closely with other web technologies.
- **Choose LangServe** if you are focused on deploying LangChain applications quickly and efficiently, prefer simplicity over flexibility, and do not require the extensive capabilities of a full-fledged web framework.

Each option has its strengths and is best suited to different types of projects and developer preferences.

## What if we wanted to build this application in a foreign language, meaning we communicate with the LLM model in a foreign language instead of in English?
* See this in 002-simpleTranslatorBuiltInSpanish.py

To modify the LangChain app so that it translates from Spanish to English and communicates with the language model (LLM) in Spanish, you'll need to make several key changes to the prompt template.

In this adjusted version:
- The `system_template` is set to ask for translations into English but is phrased in Spanish to match the language model's settings.
- The FastAPI application details have been updated to reflect the purpose of translating from Spanish to English.

This will set up an API that receives text in Spanish, asks the model to translate it into English, and then serves the English translation via an endpoint.

#### Run the server
* In terminal:
    * python 002-simpleTranslatorBuiltInSpanish.py
* Check the app in the browser:
    * [http://localhost:8000/](http://localhost:8000/)
    * Nothing fancy there... yet.

#### Now let's try the app using the "toy demo UI" provided by LangServe Playground
* Try the app in the browser:
    * [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/) 

* Enter "La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia humana." and see how it is translated into English.